In [1]:
import spacy

# Charger le modèle SpaCy par défaut
nlp = spacy.load('en_core_web_sm')


In [131]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

with open('C:\\Users\\hp\\Desktop\\StageProjet (1)\\StageProjet\\joboffers-token.json', "r") as f:
    creds = json.load(f)
    ASTRA_DB_APPLICATION_TOKEN = creds["token"]

cluster = Cluster(
    cloud={
        "secure_connect_bundle": 'C:\\Users\\hp\\Desktop\\StageProjet (1)\\StageProjet\\secure-connect-joboffers.zip',
    },
    auth_provider=PlainTextAuthProvider(
        "token",
        ASTRA_DB_APPLICATION_TOKEN,
    ),
)

session = cluster.connect()

In [122]:
import re
import nltk
from cassandra.cluster import Cluster
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from cassandra.query import SimpleStatement
from nltk.corpus import stopwords
import os
import pandas as pd
from flask import Flask,render_template,redirect,request,session
from resume_screening import resparser, match
from lxml import html





app=Flask(__name__)

os.makedirs(os.path.join(app.instance_path, 'resume_files'), exist_ok=True)

keyword=''
location=''
InternType=''
app.secret_key = 'secret'



# Connexion à Cassandra
cluster = Cluster(['127.0.0.1'])
session_cassandra = cluster.connect('job_scraping')

# Initialiser le driver Selenium
service = Service(r'C:\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe')
driver = webdriver.Chrome(service=service)

# Créer ou utiliser la keyspace
session_cassandra.execute("""
    CREATE KEYSPACE IF NOT EXISTS job_scraping 
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")
session_cassandra.set_keyspace('job_scraping')

# Créer les tables si elles n'existent pas
session_cassandra.execute("""
    CREATE TABLE IF NOT EXISTS job_offers (
        title TEXT,
        company TEXT,
        location TEXT,
        link TEXT PRIMARY KEY
    )
""")
session_cassandra.execute("""
    CREATE TABLE IF NOT EXISTS job_details (
        title TEXT,
        company TEXT,
        location TEXT,
        link TEXT PRIMARY KEY,
        description TEXT,
        min_salary INT,
        max_salary INT,
        contract_type TEXT
    )
""")

# Initialiser Selenium WebDriver
service = Service(r'C:\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe')
driver = webdriver.Chrome(service=service)

# Fonction pour effectuer le scrapping d'Indeed
def scrape_indeed():
    print("Scrapping des offres Indeed en cours...")
    keyword = 'informatique'
    location = 'France'
    url = f'https://fr.indeed.com/jobs?q={keyword}&l={location}&lang=en'
    driver.get(url)
    time.sleep(5)

    for i in range(20):  # Limité à 20 pages
        jobs_list = driver.find_elements(By.XPATH, '//div[contains(@class, "job_seen_beacon")]')
        for job in jobs_list:
            title = job.find_element(By.XPATH, './/a[contains(@class, "jcs-JobTitle")]').text.strip()
            company = job.find_element(By.XPATH, './/span[contains(@data-testid, "company-name")]').text.strip()
            location = job.find_element(By.XPATH, './/div[contains(@data-testid, "text-location")]').text.strip()
            link = job.find_element(By.XPATH, './/a[contains(@class, "jcs-JobTitle")]').get_attribute('href')

            # Vérification et insertion
            query = "SELECT link FROM job_offers WHERE link = %s"
            existing_job = session_cassandra.execute(query, (link,)).one()
            if not existing_job:
                session_cassandra.execute("""
                    INSERT INTO job_offers (title, company, location, link) 
                    VALUES (%s, %s, %s, %s)
                """, (title, company, location, link))
                print(f"Ajout : {title}")

        try:
            next_button = driver.find_element(By.XPATH, '//a[@aria-label="Next"]')
            next_button.click()
            time.sleep(5)
        except:
            print("Pas de page suivante.")
            break

    print("Scrapping des offres Indeed terminé.")

# Fonction pour compléter les détails des offres
def fetch_job_details():
    print("Récupération des détails des offres en cours...")
    rows = session_cassandra.execute("SELECT * FROM job_offers")

    for row in rows:
        link = row.link
        try:
            driver.get(link)
            description = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "jobDescriptionText"))
            ).text.strip()

            salary_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, './/span[contains(@class, "css-19j1a75 eu4oa1w0")]'))
            )
            salary = salary_element.text.strip() if salary_element else "N/A"
            
            # Fonction pour extraire les salaires min et max sous forme de nombres
            def extract_salary_range(salary_text):
                # Remplacer les espaces insécables et supprimer les caractères non numériques sauf '-'
                salary_text = salary_text.replace("\u202f", " ").replace("€", "").replace(" ", "")
                
                # Trouver tous les nombres dans le texte (gestion des salaires avec plages)
                salary_numbers = re.findall(r'\d+', salary_text)
                if len(salary_numbers) >= 2:
                    # Convertir en entiers et assigner min et max
                    min_salary = int(salary_numbers[0])
                    max_salary = int(salary_numbers[1])
                    return min_salary, max_salary
                elif len(salary_numbers) == 1:
                    # Si un seul nombre est trouvé, supposer qu'il est à la fois min et max
                    return int(salary_numbers[0]), int(salary_numbers[0])
                else:
                    # Si aucun nombre n'est trouvé, retourner None
                    return None, None

            # Utiliser la fonction pour extraire min et max
            min_salary, max_salary = extract_salary_range(salary)

            print("Salaire minimum:", min_salary)
            print("Salaire maximum:", max_salary)

            contrat_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, './/span[contains(@class, "css-k5flys eu4oa1w0")]'))
            )
            contract_type= contrat_element.text.strip() if contrat_element else "N/A"

            print("contrat",contract_type) 

            session_cassandra.execute("""
                INSERT INTO job_details (title, company, location, link, description, min_salary, max_salary, contract_type) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, (row.title, row.company, row.location, link, description, min_salary, max_salary, contract_type))
            print(f"Détails ajoutés pour : {row.title}")
        except Exception as e:
            print(f"Erreur pour {row.title} : {e}")

    print("Récupération des détails terminée.")

"""@app.before_first_request
def initialize_scraping():
    scrape_indeed()
    fetch_job_details()"""
    
@app.route('/')
def index():
    
    return render_template("index.html")

@app.route("/uploadInit")
def uploadInit():
    global InternType
    InternType='Init'
    print(InternType)
    return render_template("upload.html")

@app.route("/home")
def home():
    return redirect('/')



@app.route('/submit', methods=['POST'])
def submit_data():
    global cvPath 
    global InternType
    if request.method == 'POST':        
        f = request.files['userfile']
        cvPath = os.path.join(app.instance_path, 'resume_files', f.filename)
        f.save(cvPath)
        
    stopw  = set(stopwords.words('french'))  # Utilisation des stopwords en français
    
    # Charger les données depuis la base Cassandra plutôt que depuis un fichier CSV
    query = "SELECT * FROM job_details"
    rows = session_cassandra.execute(query)
    job = pd.DataFrame(rows)  # Conversion des résultats de Cassandra en DataFrame
    
    job['test'] = job['description'].apply(lambda x: ' '.join([word for word in str(x).split() if len(word) > 2 and word not in stopw]))
    df = job.drop_duplicates(subset='test').reset_index(drop=True)
    df['clean'] = df['test'].apply(match.preprocessing)  # Assurez-vous que la fonction de pré-traitement fonctionne avec le français
    jobdesc = df['clean'].values.astype('U')  # Conversion des descriptions en tableau numpy
    
    # Extraction des compétences depuis le CV
    skills = resparser.skill(f'instance/resume_files/{f.filename}')
    skills.append(match.preprocessing(skills[0]))
    del skills[0]

    # Calcul de la similarité entre compétences et descriptions d'emploi
    count_matrix = match.vectorizing(skills[0], jobdesc)
    matchPercentage = match.coSim(count_matrix)
    matchPercentage = pd.DataFrame(matchPercentage, columns=['Skills Match'])

    # Recommandations d'offres d'emploi basées sur la similarité des compétences
    result_cosine = df[['title', 'company', 'link','contract_type','location']]  # Assurez-vous que les noms de colonnes sont corrects
    result_cosine = result_cosine.join(matchPercentage)
    result_cosine = result_cosine[['title', 'company', 'Skills Match', 'link','contract_type']]
    result_cosine.columns = ['Titre de l\'emploi', 'Entreprise', 'Correspondance des compétences', 'Lien','contrat']
    result_cosine = result_cosine.sort_values('Correspondance des compétences', ascending=False).reset_index(drop=True).head(20)
    
    print(result_cosine)
    return render_template('upload.html', column_names=result_cosine.columns.values, row_data=list(result_cosine.values.tolist()),
                           link_column="Lien", zip=zip)

@app.route('/submit2', methods=['POST'])
def submit_data_filtre():
    if request.method == 'POST':
        print(cvPath)
        # Récupération des filtres
        location_filter = request.form.get('location', '').strip()
        contract_filter = request.form.get('contract', '').strip()
        salary_filter = request.form.get('min_salary', '').strip()

        stopw = set(stopwords.words('french'))

        # Chargement des données de Cassandra avec filtres
        query = "SELECT * FROM job_details"
        filters = []
        
       
        
        if salary_filter and salary_filter.startswith(">"):
            try:
                min_salary = int(salary_filter[1:])
                filters.append(f"min_salary >= {min_salary}")
            except ValueError:
                pass

        if filters:
            query += " WHERE " + " AND ".join(filters)
        
        print(query)
        rows = session_cassandra.execute(query)
        job = pd.DataFrame(rows)
        if contract_filter:
            cf=f'{contract_filter}'
            job2=job[job['contract_type'].str.contains(cf)==True]
            job=job2
        if location_filter:
           lc=f'{location_filter}'
           job1=job[job['location'].str.contains(lc)==True]
           job=job1
        job['test'] = job['description'].apply(lambda x: ' '.join([word for word in str(x).split() if len(word) > 2 and word not in stopw]))
        df = job.drop_duplicates(subset='test').reset_index(drop=True)
        df['clean'] = df['test'].apply(match.preprocessing)
        jobdesc = df['clean'].values.astype('U')

        skills = resparser.skill(cvPath)
        skills.append(match.preprocessing(skills[0]))
        del skills[0]

        count_matrix = match.vectorizing(skills[0], jobdesc)
        matchPercentage = match.coSim(count_matrix)
        matchPercentage = pd.DataFrame(matchPercentage, columns=['Skills Match'])

        result_cosine = df[['title', 'company', 'link', 'contract_type','location']]
        result_cosine = result_cosine.join(matchPercentage)
        result_cosine = result_cosine[['title', 'company', 'Skills Match', 'link', 'contract_type','location']]
        result_cosine.columns = ['Titre de l\'emploi', 'Entreprise', 'Correspondance des compétences', 'Lien', 'contrat','location']
        result_cosine = result_cosine.sort_values('Correspondance des compétences', ascending=False).reset_index(drop=True).head(20)

        return render_template('upload.html', column_names=result_cosine.columns.values, row_data=list(result_cosine.values.tolist()),
                               link_column="Lien", zip=zip)


if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
127.0.0.1 - - [26/Nov/2024 16:25:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 16:25:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 16:25:24] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 16:25:24] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 16:25:24] "GET /static/js/script.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 16:25:24] "GET /static/js/script.js HTTP/1.1" 304 -
127.0.0.1 - - [26/Nov/2024 16:25:24] "GET /static/images/university.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 16:25:24] "GET /static/images/university.png HTTP/1.1" 304 -
127.0.0.1 - - [26/Nov/2024 16:25:24] "GET /static/images/pfa.png HTTP

Init


127.0.0.1 - - [26/Nov/2024 16:25:31] "GET /static/css/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 16:25:31] "GET /static/css/style.css HTTP/1.1" 304 -


In [120]:
query1="CREATE INDEX index_nom ON job_details(contract_type)"
session_cassandra.execute(query1)

In [110]:
cluster = Cluster(['127.0.0.1'])
session_cassandra = cluster.connect('job_scraping')
query = "SELECT * FROM job_details"

print(query)

SELECT * FROM job_details


In [118]:
stopw  = set(stopwords.words('french'))  # Utilisation des stopwords en français
# Connexion à Cassandra
cluster = Cluster(['127.0.0.1'])
session = cluster.connect('job_scraping')  
# Charger les données depuis la base Cassandra plutôt que depuis un fichier CSV
query = "SELECT * FROM job_details "
rows = session.execute(query)
job = pd.DataFrame(rows)  # Conversion des résultats de Cassandra en DataFrame

job

InvalidRequest: Error from server: code=2200 [Invalid query] message="unconfigured table job_details"

In [12]:
stopw  = set(stopwords.words('french'))  # Utilisation des stopwords en français
# Connexion à Cassandra
cluster = Cluster(['127.0.0.1'])
session_cassandra = cluster.connect('job_scraping') 
# Charger les données depuis la base Cassandra plutôt que depuis un fichier CSV
query = "SELECT * FROM job_details"
rows = session_cassandra.execute(query)
job = pd.DataFrame(rows)  # Conversion des résultats de Cassandra en DataFrame
    
job['test'] = job['description'].apply(lambda x: ' '.join([word for word in str(x).split() if len(word) > 2 and word not in stopw]))
df = job.drop_duplicates(subset='test').reset_index(drop=True)
df['clean'] = df['test'].apply(match.preprocessing)  # Assurez-vous que la fonction de pré-traitement fonctionne avec le français
jobdesc = df['clean'].values.astype('U')  # Conversion des descriptions en tableau numpy
    
# Extraction des compétences depuis le CV
skills = resparser.skill("SAIDA_ABARARH_CV_Resume.pdf")
skills.append(match.preprocessing(skills[0]))
del skills[0]

print(skills)

2024-11-17 21:31:54,136 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to C:\Users\hp\AppData\Local\Temp\tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to C:\Users\hp\AppData\Local\Temp\tika-server.jar.
2024-11-17 21:31:55,626 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to C:\Users\hp\AppData\Local\Temp\tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to C:\Users\hp\AppData\Local\Temp\tika-server.jar.md5.
2024-11-17 21:31:56,733 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2024-11-17 21:32:01,747 [MainTh

['ia et data science  machine learning deep learning intelligence artificielle pytorch tensorflow llm prompt engineering rag finetuning opencv langchain • gcp aws azure• docker git unix• sports déquipe• bénévolat dans une association• participation à des hackathons•']


In [15]:
# Calcul de la similarité entre compétences et descriptions d'emploi
count_matrix = match.vectorizing(skills[0], jobdesc)
matchPercentage = match.coSim(count_matrix)
matchPercentage = pd.DataFrame(matchPercentage, columns=['Skills Match'])
matchPercentage


,Skills Match
0,0.00
1,3.57
2,2.73
3,1.34
4,0.00
5,2.08
6,2.11
7,2.33
8,1.17
9,0.00


In [20]:
# Recommandations d'offres d'emploi basées sur la similarité des compétences
result_cosine = df[['title', 'company', 'link']]  # Assurez-vous que les noms de colonnes sont corrects
result_cosine = result_cosine.join(matchPercentage)
result_cosine = result_cosine[['title', 'company', 'Skills Match', 'link']]
result_cosine.columns = ['Titre de l\'emploi', 'Entreprise', 'Correspondance des compétences', 'Lien']
result_cosine = result_cosine.sort_values('Correspondance des compétences', ascending=False).reset_index(drop=True).head(20)
result_cosine

,Titre de l'emploi,Entreprise,Correspondance des compétences,Lien
0,Technicien Informatique (H/F),Groupe MARTY,6.16,https://fr.indeed.com/rc/clk?jk=aeffaa093009b4...
1,ACHETEUR MARCHES PUBLICS INFORMATIQUE ET TELEC...,UGAP,3.57,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...
2,Technicien informatique H/F,GROUPE JANNEAU,2.73,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...
3,Ingénieur Système Infrastructure Stockage Open...,EURO-INFORMATION PRODUCTION,2.33,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...
4,Technicien support informatique H/F,SartheFIBRE,2.11,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...
5,Technicien informatique H/F,HAD 3G SANTE - HOME SANTE,2.08,https://fr.indeed.com/rc/clk?jk=d73da8a13f0836...
6,Chef de Projet Informatique H/F - Pilotez des ...,Avisto,1.34,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...
7,Technicien de maintenance automatismes et info...,RTE,1.17,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...
8,Acheteur confirmé IT/Système d’informations F/...,RATP,0.94,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...
9,Chargé de Support Réglementaire CEE (H/F),Economie d'energie,0.64,https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYl...


In [5]:
import os
import pandas as pd
from flask import Flask,render_template,redirect,request,session
import linkedin_web_scraping as linkd
import indeed_web_scraping as ind
from resume_screening import resparser, match
from lxml import html

import nltk

from nltk.corpus import stopwords
stopw  = set(stopwords.words('english'))

job = pd.read_csv('indeed_data.csv')
job['test'] = job['description'].apply(lambda x: ' '.join([word for word in str(x).split() if len(word)>2 and word not in (stopw)]))
df = job.drop_duplicates(subset='test').reset_index(drop=True)
df['clean'] = df['test'].apply(match.preprocessing)
jobdesc = (df['clean'].values.astype('U'))   
jobdesc 

array(['job description life grab grab every grabber guided the grab way spells mission believe achieve it operating principles hs heart hunger honour humility these principles guide help make decisions work create economic empowerment people southeast asia get know team the deliveries analytics team responsible growing optimizing grabfood grabmart grabexpress businesses intelligent use data focus providing datadriven insights to deliver impeccable seamless experience consumers develop allocationrelated features ensure ontime infull delivery consumers ensuring optimal utilization driverpartners helping earn living develop products enable merchantpartners maximize sales platform partner product business teams launch new innovative initiatives grabfood grabmart andor grabexpress cater needs consumers get know role looking data scientist analytics provide critical insights drive grabmart dark stores business line product goals forward indonesia this requires working closely various teams 

In [7]:
skills = resparser.skill(resume_file='CV.pdf')
skills.append(match.preprocessing(skills[0]))
del skills[0]

['learning big data machine learning sql python deep learning']

In [9]:
skills[0]

'learning big data machine learning sql python deep learning'

In [11]:
count_matrix = match.vectorizing(skills[0], jobdesc)
matchPercentage = match.coSim(count_matrix)
matchPercentage = pd.DataFrame(matchPercentage, columns=['Skills Match'])

matchPercentage

,Skills Match
0,8.70
1,20.07
2,14.66
3,9.43
4,20.58
...,...
562,8.26
563,9.67
564,5.62
565,22.48


In [13]:
df

,title,company,salary,description,link,test,clean
0,Data Scientist (Analytics),Grab,NaN,"Job Description: Life at Grab At Grab, every G...",https://id.indeed.com/rc/clk?jk=acd6479660d3fc...,"Job Description: Life Grab Grab, every Grabber...",job description life grab grab every grabber g...
1,Data Scientist,Sayurbox,NaN,Job Description Be a part of Sayurbox data tea...,https://id.indeed.com/rc/clk?jk=40482050a5e4d8...,Job Description part Sayurbox data team target...,job description part sayurbox data team target...
2,Senior Data Scientist - GoPay,GO-JEK,NaN,Location Jakarta Work Type Permanent Applicati...,https://id.indeed.com/rc/clk?jk=60463a42bb19a6...,Location Jakarta Work Type Permanent Applicati...,location jakarta work type permanent applicati...
3,Data Scientist,Allianz Indonesia,NaN,Bachelor Degree with field of study mathematic...,https://id.indeed.com/rc/clk?jk=835f2c980867fe...,"Bachelor Degree field study mathematic, inform...",bachelor degree field study mathematic informa...
4,Senior Data Scientist,Sayurbox,NaN,Job Description: Build Sayurbox data team acco...,https://id.indeed.com/rc/clk?jk=da6afcc2ee471d...,Job Description: Build Sayurbox data team acco...,job description build sayurbox data team accor...
...,...,...,...,...,...,...,...
562,Cross-border Business Analysis Intern,XINGYUN GROUP,NaN,l JOB DESCRIPTIONS Ø Market and Competitive Dy...,https://id.indeed.com/company/XingYun-Group/jo...,JOB DESCRIPTIONS Market Competitive Dynamics A...,job descriptions market competitive dynamics a...
563,Senior Business Intelligence Engineer,Home Credit,NaN,IT and Software Minimum Qualifications Bachelo...,https://id.indeed.com/rc/clk?jk=788cd35555386d...,Software Minimum Qualifications Bachelor degre...,software minimum qualifications bachelor degre...
564,Market Intelligence Researcher,GO-JEK,NaN,Location Jakarta Work Type Permanent Applicati...,https://id.indeed.com/rc/clk?jk=36c04039c9ed1f...,Location Jakarta Work Type Permanent Applicati...,location jakarta work type permanent applicati...
565,Business Intelligence Developer E-Commerce,Kompas Gramedia,NaN,Learning and fully understanding the data land...,https://id.indeed.com/rc/clk?jk=e8069d833cf7db...,Learning fully understanding data landscape da...,learning fully understanding data landscape da...


In [17]:
#Job Vacancy Recommendations
result_cosine = df[['title','company','link']]
result_cosine = result_cosine.join(matchPercentage)
result_cosine = result_cosine[['title','company','Skills Match','link']]
result_cosine.columns = ['title','company','Skills Match','link']
result_cosine = result_cosine.sort_values('Skills Match', ascending=False).reset_index(drop=True).head(20)
result_cosine

,title,company,Skills Match,link
0,Data Scientist,PGI Data,50.93,https://id.indeed.com/rc/clk?jk=c3c6e49115655c...
1,"Data Scientist - MLE Ops (Loyalty, Partnership...",GO-JEK,47.00,https://id.indeed.com/rc/clk?jk=fa03fc600b227f...
2,Data Scientist,PT Majoo Teknologi Indonesia,45.61,https://id.indeed.com/rc/clk?jk=6565c5614b8e80...
3,Data Scientist,FinAccel,43.60,https://id.indeed.com/rc/clk?jk=a6d93e0bf16a05...
4,Data Scientist - QuantumBlack,McKinsey & Company,35.45,https://id.indeed.com/rc/clk?jk=d04c544ef713f7...
5,Senior Data Scientist - Research,GO-JEK,33.80,https://id.indeed.com/rc/clk?jk=6fed623a8a3dc6...
6,Data Scientist,Cybertrend,32.87,https://id.indeed.com/rc/clk?jk=fb3884ea1cedba...
7,Sr Machine Learning Engineer,Global Tiket Network,32.60,https://id.indeed.com/rc/clk?jk=d83aebce84b875...
8,Data Scientist - Demand Generation,GO-JEK,30.96,https://id.indeed.com/rc/clk?jk=6988f3df6b3b41...
9,Data Engineer - Gramedia Digital,Kompas Gramedia,30.11,https://id.indeed.com/rc/clk?jk=b52d2fcd5a0280...
